# Battery bidding functions

## To do

This code represents a battery which is participating in a real-time market (every 5min). At the beginning of each market interval, the battery places a buy bid (maximum price at which it is willing to buy) and a supply bid (minimum price at which it is willing to sell). Then, the market reveals the true real-time price. If the price is lower than the buy bid, the battery charges. If the price is higher than the sell price, the battery discharges. 

The objective of this task is to write bidding functions which maximize the battery's profit or some other objective.

Please go through the following steps:

Week 3:
* Go through the code and try to understand each part. Currently, a very simple bidding function (simple_bid()) is implemented. Run the code from the top.
* Write the function calculate_battery_profit() which calculates the profits from operating the battery.
* Write the function visualize_battery_dispatch() which visualizes when the battery dispatches as well as the market price.
* Test your code for a week-long simulation period.

Week 4:
* Think of other evaluations and visualizations of the battery behavior and implement them.
* Make a strategy of how to implement one of the following new bidding functions and discuss them with Marie-Louise and Lynne:
    - bid_costminizing()
    - bid_optimal_autarky()
    - bid_optimal_RL()
* Implement your bidding function and test and visualize it.

Week 5:
* Repeat the same for the other bidding functions.

Once you have finalized your code, we will implement your code in GridLAB-D and test them in a more complex distribution system. For the next steps, have a look at the GoogleDoc with your workplan-

When you have questions, don't hesitate to contact Anna, Lynne, or Marie-Louise!

## Modules

In [107]:
import pandas
pandas.options.mode.chained_assignment = None
pandas.set_option('display.max_rows', 50)
import cvxpy
from matplotlib.dates import drange
import datetime

## Define bidding functions

In [108]:
df_battery_state
# change order possibly?

,battery_name,house_name,SOC_min,SOC_max,i_max,u_max,efficiency,SOC_t,active_t-1,active_t,threshold_sell,threshold_buy,q_sell,p_sell,q_buy,p_buy,residual_s,residual_b
0,batt_1,house_1,1.0,5.0,1.0,1.0,0.99,7.150519,0,-1,0.0,0.0,1.0,16.81509,0.0,16.480469,73.806,-26.406


In [109]:
def bid_simple(dt_sim_time,df_battery_state,df_WS,df_prices):
    prec = 3 # rounding
    
    # Price bid
    # mean_p is the mean price starting from dt_sim_time to 4 hours in the future
    mean_p = df_WS.loc[dt_sim_time:(dt_sim_time+datetime.timedelta(hours = 4))]['DA'].mean()
    df_battery_state['p_sell'] = mean_p / df_battery_state['efficiency']
    df_battery_state['p_buy'] = mean_p * df_battery_state['efficiency']
    
    # Quantity depends on SOC and u
    safety_fac = 0.99
    df_battery_state['residual_s'] = round((3600./interval)*(df_battery_state['SOC_t'] - df_battery_state['SOC_min']),prec) #Recalculate to kW
    #print(df_battery_state[['residual_s','u_max']])
    df_battery_state['q_sell'] = df_battery_state[['residual_s','u_max']].min(axis=1) #in kW / only if fully dischargeable
    df_battery_state['q_sell'].loc[df_battery_state['q_sell'] < 0.1] = 0.0
    df_battery_state['residual_b'] = round((3600./interval)*(safety_fac*df_battery_state['SOC_max'] - df_battery_state['SOC_t']),prec) #Recalculate to kW
    #print(df_battery_state[['residual_b','u_max']])
    df_battery_state['q_buy'] = df_battery_state[['residual_b','u_max']].min(axis=1) #in kW
    df_battery_state['q_buy'].loc[df_battery_state['q_buy'] < 0.1] = 0.0
    
    #print(df_battery_state)
    return df_battery_state

In [110]:
def bid_costminizing(df_battery_state): # Adjust input
    time_horizon = 4*3600/interval # optimization horizon
    # Determine the optimal dispatch schedule of the battery over the optimization horizon,
    #    given day-ahead (DA) prices from df_WS. Use the package cvxpy.
    # Determine profits from (t+1) on if battery had charged/discharged/not done anything in t
    # The profit/cost from charging/doing nothing/discharging in t and the future value in (t+1) 
    #    if the value of the action charging/doing nothing/discharging in t
    #    --> This principle is called the Bellman principle
    # Which prices make the battery operator equivalent between:
    # - charging or doing nothing? --> This is the maximum price at which the battery operator
    #   is willing to buy
    # - discharging or doing nothing? --> This is the minimum price at which the battery operator
    #   is willing to sell
    return df_battery_state

In [111]:
def bid_optimal_autarky(df_battery_state):
    # Design and implement a bidding strategy of an agent who values a high state-of-charge
    # but is willing to go below the minimum SOC if the agent could achieve substantial profits
    return df_battery_state

In [112]:
def bid_optimal_RL(df_battery_state):
    # Maximize battery profit using Reinforcement Learning
    # Use the following RL algorithm: https://dl.acm.org/doi/10.1145/3208903.3208907 (Section 4)
    return df_battery_state

## Evaluation functions

In [128]:
def calculate_battery_profit(df_awarded_bids):
    # Calculate profit from selling and buying energy
    # prices are in USD/MWh and u_max = 1 kWh
    # Multiply by 1/12 because market intervals are 5 minutes 
    profit=0
    money_spent=0
    money_made=0
    for index, row in df_awarded_bids.iterrows():
        if row['S_D']=='S':
            money_made = money_made + (row['bid_price'])*(1/12)*(1/1000)
        elif row['S_D']=='D':
            money_spent = money_spent + (row['bid_price'])*(1/12)*(1/1000)
    profit=money_made-money_spent      
    return profit

In [114]:
def visualize_battery_dispatch():
    # Visualize battery dispatch over time as well as the market price
    return

## Other functions

In [115]:
# Update battery function: updates the batteries' state of charge
# (Same as in market_simulation code)
def update_battery(df_battery_state):
    #-1: discharging, 0 no activity, 1 charging
    #history is saved by battery recorder (P_out)
    df_battery_state['active_t-1'] = df_battery_state['active_t']
    df_battery_state['active_t'] = 0 
    df_battery_state['SOC_t'] = df_battery_state['SOC_t']*df_battery_state['efficiency']
    df_battery_state['SOC_t'].loc[df_battery_state['active_t-1'] == -1] += interval/3600*df_battery_state['q_sell'].loc[df_battery_state['active_t-1'] == -1]
    df_battery_state['SOC_t'].loc[df_battery_state['active_t-1'] == 1] += interval/3600*df_battery_state['q_buy'].loc[df_battery_state['active_t-1'] == 1]
    return df_battery_state

In [116]:
# Write bids to result tables
def submit_bids_battery(dt_sim_time,retail,df_bids,df_supply_bids,df_buy_bids):
    for ind in df_bids.index:
        if df_bids['q_sell'].loc[ind] > 0.0:
            #retail.sell(df_bids['q_sell'].loc[ind],df_bids['p_sell'].loc[ind],gen_name=ind)
            df_supply_bids = df_supply_bids.append(pandas.DataFrame(columns=df_supply_bids.columns,data=[[dt_sim_time,ind,float(df_bids['p_sell'].loc[ind]),float(df_bids['q_sell'].loc[ind])]]),ignore_index=True)
        if df_bids['q_buy'].loc[ind] > 0.0:
            #retail.buy(df_bids['q_buy'].loc[ind],df_bids['p_buy'].loc[ind],active=df_bids['active_t-1'].loc[ind],appliance_name=ind)
            df_buy_bids = df_buy_bids.append(pandas.DataFrame(columns=df_buy_bids.columns,data=[[dt_sim_time,ind,float(df_bids['p_buy'].loc[ind]),float(df_bids['q_buy'].loc[ind])]]),ignore_index=True)
    df_bids['active_t-1'] = 0
    return retail,df_supply_bids,df_buy_bids

In [117]:
# Determines if battery should be dispatched and how
def set_battery_by_price(dt_sim_time,df_bids_battery,Pd, df_awarded_bids):
    #Determine activity
    df_bids_battery.at[:,'active_t'] = 0
    df_bids_battery.at[(df_bids_battery['p_buy'] >= Pd) & (df_bids_battery['SOC_t'] < df_bids_battery['SOC_max']),'active_t'] = 1
    df_bids_battery.at[(df_bids_battery['p_sell'] <= Pd) & (df_bids_battery['SOC_t'] > 0.0),'active_t'] = -1
    #print(df_bids_battery)
    # Save to awarded bids
    for battery in df_bids_battery.index:
        batt_number = battery
        SOC = df_bids_battery['SOC_t'].loc[battery] #this is SOC at the beginning of the period t
        active = df_bids_battery['active_t'].loc[battery] #this is activity in t
        if active == 1:
            q_bid = df_bids_battery['q_buy'].loc[battery]
            p_bid = df_bids_battery['p_buy'].loc[battery]
            df_awarded_bids = df_awarded_bids.append(pandas.DataFrame(columns=df_awarded_bids.columns,data=[[dt_sim_time,battery,float(p_bid),float(q_bid),'D']]),ignore_index=True)
        elif active == -1:
            q_bid = df_bids_battery['q_sell'].loc[battery]
            p_bid = df_bids_battery['p_sell'].loc[battery]
            df_awarded_bids = df_awarded_bids.append(pandas.DataFrame(columns=df_awarded_bids.columns,data=[[dt_sim_time,battery,float(p_bid),float(q_bid),'S']]),ignore_index=True)
    return df_bids_battery,df_awarded_bids

## Initialize objects and data

In [118]:
# Initialize battery 1
cols_battery = ['battery_name','house_name','SOC_min','SOC_max','i_max','u_max','efficiency','SOC_t','active_t-1','active_t','threshold_sell','threshold_buy','q_sell','p_sell','q_buy','p_buy']
values = [['batt_1','house_1',1.,5.,1.,1.,0.99,2.5,0,0,0.0,0.0,0.0,0.0,0.0,0.0]]
df_battery_state = pandas.DataFrame(columns=cols_battery,data=values)

In [119]:
# WS costs
df_WS = pandas.read_csv('glm_generation_Austin/Ercot_LZ_SOUTH.csv',parse_dates=True,index_col=[0]) # For year 2016
df_WS.head()

,DA,RT
2016-01-01 00:00:00,18.22,15.48
2016-01-01 00:05:00,18.22,15.48
2016-01-01 00:10:00,18.22,15.48
2016-01-01 00:15:00,18.22,15.33
2016-01-01 00:20:00,18.22,15.33


In [120]:
# Initialize market parameters
start = datetime.datetime( 2016, 7, 1)
end = datetime.datetime( 2016, 7, 1, 1, 0)
interval = 300 # interval of market operation
market_intervals = pandas.date_range(start,end,freq=str(int(interval/60))+'min')
market_intervals #the first hour in 5 minute increments

DatetimeIndex(['2016-07-01 00:00:00', '2016-07-01 00:05:00',
               '2016-07-01 00:10:00', '2016-07-01 00:15:00',
               '2016-07-01 00:20:00', '2016-07-01 00:25:00',
               '2016-07-01 00:30:00', '2016-07-01 00:35:00',
               '2016-07-01 00:40:00', '2016-07-01 00:45:00',
               '2016-07-01 00:50:00', '2016-07-01 00:55:00',
               '2016-07-01 01:00:00'],
              dtype='datetime64[ns]', freq='5T')

## Run market

In [121]:
# Set up df to save results
# Setting up empty data frames
df_prices = pandas.DataFrame(columns=['clearing_price','clearing_quantity','unresponsive_loads'])
df_buy_bids = pandas.DataFrame(columns=['timestamp','appliance_name','bid_price','bid_quantity'])
df_supply_bids = pandas.DataFrame(columns=['timestamp','appliance_name','bid_price','bid_quantity'])
df_awarded_bids = pandas.DataFrame(columns=['timestamp','appliance_name','bid_price','bid_quantity','S_D'])

# Run market
for dt_sim_time in market_intervals:
    #print(dt_sim_time)
    # Update physical state of battery
    df_battery_state = update_battery(df_battery_state)
    
    # Setup market
    retail = None # this is a placeholder
    
    # Bid : THIS IS WHERE YOU SHOULD IMPLEMENT AND TEST BIDDING FUNCTIONS
    df_battery_state = bid_simple(dt_sim_time,df_battery_state,df_WS,df_prices)
    #bid_costminizing(dt_sim_time,df_battery_state,df_WS,df_prices)
    #bid_optimal_autarky(dt_sim_time,df_battery_state,df_WS,df_prices)
    retail,df_supply_bids,df_buy_bids = submit_bids_battery(dt_sim_time,retail,df_battery_state,df_supply_bids,df_buy_bids)
    
    # Clear market and save price
    Pd = df_WS['RT'].loc[dt_sim_time:dt_sim_time].iloc[-1]
    Qd = max(df_battery_state['q_sell'].loc[df_battery_state['p_sell'] <= Pd].sum(),df_battery_state['q_buy'].loc[df_battery_state['p_buy'] >= Pd].sum())
    unresp_load = 0.0
    df_temp = pandas.DataFrame(index=[dt_sim_time],columns=['clearing_price','clearing_quantity','unresponsive_loads'],data=[[Pd,Qd,unresp_load]])
    df_prices = df_prices.append(df_temp)
    
    # Dispatch battery
    # Calls the function in order to append and fill the empty data frames
    df_bids_battery,df_awarded_bids = set_battery_by_price(dt_sim_time,df_battery_state,Pd, df_awarded_bids)
    
    #print(df_battery_state)
    #set_battery_by_price
  

## Visualize market results

In [122]:
# buy bids
# discrepancy with amount of rows shown
df_buy_bids

,timestamp,appliance_name,bid_price,bid_quantity
0,2016-07-01 00:00:00,0,17.274288,1.0
1,2016-07-01 00:05:00,0,17.205392,1.0
2,2016-07-01 00:10:00,0,17.136496,1.0
3,2016-07-01 00:15:00,0,17.067600,1.0
4,2016-07-01 00:20:00,0,16.998704,1.0
5,2016-07-01 00:25:00,0,16.929808,1.0
6,2016-07-01 00:30:00,0,16.860912,1.0
7,2016-07-01 00:35:00,0,16.792016,1.0
8,2016-07-01 00:40:00,0,16.723120,1.0
9,2016-07-01 00:45:00,0,16.654224,1.0


In [101]:
# supply bids
df_supply_bids

,timestamp,appliance_name,bid_price,bid_quantity
0,2016-07-01 00:00:00,0,17.625026,1.0
1,2016-07-01 00:05:00,0,17.554731,1.0
2,2016-07-01 00:10:00,0,17.484436,1.0
3,2016-07-01 00:15:00,0,17.414141,1.0
4,2016-07-01 00:20:00,0,17.343847,1.0
5,2016-07-01 00:25:00,0,17.273552,1.0
6,2016-07-01 00:30:00,0,17.203257,1.0
7,2016-07-01 00:35:00,0,17.132962,1.0
8,2016-07-01 00:40:00,0,17.062667,1.0
9,2016-07-01 00:45:00,0,16.992373,1.0


In [103]:
# awarded bids
#pandas.set_option('display.max_rows', 50)
df_awarded_bids
#df_awarded_bids.iloc[0:50]

,timestamp,appliance_name,bid_price,bid_quantity,S_D
0,2016-07-01 00:00:00,0,17.625026,1.0,S
1,2016-07-01 00:05:00,0,17.554731,1.0,S
2,2016-07-01 00:10:00,0,17.484436,1.0,S
3,2016-07-01 00:15:00,0,17.414141,1.0,S
4,2016-07-01 00:20:00,0,17.343847,1.0,S
5,2016-07-01 00:25:00,0,17.273552,1.0,S
6,2016-07-01 00:30:00,0,17.203257,1.0,S
7,2016-07-01 00:35:00,0,17.132962,1.0,S
8,2016-07-01 00:40:00,0,17.062667,1.0,S
9,2016-07-01 00:45:00,0,16.992373,1.0,S


In [104]:
# market result
df_prices

,clearing_price,clearing_quantity,unresponsive_loads
2016-07-01 00:00:00,19.48,1.0,0.0
2016-07-01 00:05:00,19.48,1.0,0.0
2016-07-01 00:10:00,19.48,1.0,0.0
2016-07-01 00:15:00,19.92,1.0,0.0
2016-07-01 00:20:00,19.92,1.0,0.0
2016-07-01 00:25:00,19.92,1.0,0.0
2016-07-01 00:30:00,19.88,1.0,0.0
2016-07-01 00:35:00,19.88,1.0,0.0
2016-07-01 00:40:00,19.88,1.0,0.0
2016-07-01 00:45:00,19.72,1.0,0.0


## Calculate battery profit

In [129]:
# In USD
calculate_battery_profit(df_awarded_bids)

0.01863966192537623

In [29]:
visualize_battery_dispatch()